<a href="https://colab.research.google.com/github/carlosdgerez/machine_learning/blob/main/preprocesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd

housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
housing.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,...,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,...,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,...,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,...,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,...,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0


In [17]:
#Changing date to be in the same format as the rest of the years features
import datetime
housing['date'] = pd.DatetimeIndex(housing['date']).year



In [18]:
# selecting clean dat including lat and long
clean_data = housing.drop(['id'], axis = 1)


In [19]:

# change date name to something more descriptive
clean_data.rename(columns={'date': 'sale_date'}, inplace=True)


In [4]:
# get a training set: a X as with the features to relate and a y with the  target 
X = clean_data.iloc[:,1:-1]
y = clean_data.price


In [5]:
# Start spliting the data and import libraries
# this time will use the optimal parameters from the previous run it grid search
#import train split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.30, random_state=1)


In [6]:
#  xgboost regresor
import xgboost
from sklearn import metrics as sm

# createn instance and fit the model

clf2 = xgboost.XGBRegressor(random_state=1, max_depth= 3, n_estimators= 900, learning_rate= 0.1, verbosity = 0).fit(X_train, y_train)
y_pred = clf2.predict(X_test)

# This is how to generate a regresion report ( for regresion )
print("Mean absolute error =", round(
    sm.mean_absolute_error(y_test, y_pred), 2))
print("Mean squared error =", round(
    sm.mean_squared_error(y_test, y_pred), 2))
print("Median absolute error =", round(
    sm.median_absolute_error(y_test, y_pred), 2))
print("Explain variance score =", round(
    sm.explained_variance_score(y_test, y_pred), 2))
print("R2 score =", sm.r2_score(y_test, y_pred))
print( "ROOT Mean squared error = ", 
    sm.mean_squared_error(y_test, y_pred, squared=False 
))


Mean absolute error = 66140.31
Mean squared error = 13498678662.75
Median absolute error = 41091.7
Explain variance score = 0.9
R2 score = 0.8988006414247878
ROOT Mean squared error =  116183.81411689393


In [20]:
# download the income data

zip_code = pd.read_csv('https://raw.githubusercontent.com/carlosdgerez/machine_learning/main/mean_income_washington_by_zipcode.csv')
zip_code.head()

,zip_code,mean_income
0,98001,102586
1,98002,70945
2,98003,76753
3,98004,210129
4,98005,186020


In [21]:
# rename the coluns as the one in clean data to use merge by this column
zip_code.rename(columns={'zip_code': 'zipcode'}, inplace = True)

In [22]:
# Use merge to get the data in the dataframe
new_clean_data = pd.merge(clean_data, zip_code, how= 'left', on= 'zipcode')

In [24]:
new_clean_data.head()

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,mean_income
0,2014,4,3.25,3760,4675,2.0,0,0,3,8,...,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0,134178
1,2015,3,1.75,1460,7800,1.0,0,0,2,7,...,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0,100910
2,2014,4,1.00,1340,5800,1.5,0,2,3,7,...,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0,151980
3,2014,3,1.75,1440,8050,1.0,0,0,3,8,...,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0,100910
4,2014,2,1.50,1780,4750,1.0,0,0,4,7,...,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0,166123


In [25]:
# reorder the data to have the target price at the end
column_to_move = new_clean_data.pop("mean_income")

# insert column with insert(location, column_name, column_value)

new_clean_data.insert(19, "mean_income", column_to_move)


In [26]:
new_clean_data.head()

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,mean_income,price
0,2014,4,3.25,3760,4675,2.0,0,0,3,8,...,1020,2007,0,98038,47.3862,-122.048,3280,4033,134178,429900.0
1,2015,3,1.75,1460,7800,1.0,0,0,2,7,...,420,1979,0,98023,47.3035,-122.382,1310,7865,100910,233000.0
2,2014,4,1.00,1340,5800,1.5,0,2,3,7,...,0,1914,0,98116,47.5658,-122.389,1900,5800,151980,455000.0
3,2014,3,1.75,1440,8050,1.0,0,0,3,8,...,0,1985,0,98023,47.3187,-122.390,1790,7488,100910,258950.0
4,2014,2,1.50,1780,4750,1.0,0,0,4,7,...,700,1947,0,98117,47.6859,-122.395,1690,5962,166123,555000.0
